In [1]:
import warnings
warnings.filterwarnings("ignore")

## Research various skew related topics
* **This requires plotly 4.0 or greater** *

### Analyse options volatility in futures contract, creating graphs that show vertical skew historically over time, and also show skew vs futures prices over time


In [1]:
import zipfile
import glob
import pandas as pd
import numpy as np

from argparse import ArgumentParser
from argparse import RawDescriptionHelpFormatter
import sys
import os
if  not './' in sys.path:
    sys.path.append('./')
if  not '../' in sys.path:
    sys.path.append('../')

from barchartacs import build_db
from barchartacs import db_info
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.tools as tls
from plotly.subplots import make_subplots
from plotly.graph_objs.layout import Font,Margin
from IPython import display

import datetime
import io
from tqdm import tqdm,tqdm_notebook
from barchartacs import pg_pandas as pg
import mibian
import py_vollib
import importlib
from py_vollib import black
from py_vollib.black import implied_volatility
import ipdb
import traceback
import pandas_datareader.data as pdr

# importlib.reload(build_db)

/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



### important global variables

In [ ]:
#bash barchartacs_step_03_specify_months.sh 'sep' 2020 2020 True local

In [2]:

DEBUG_IT=False
opttab = 'sec_schema.options_table'
futtab = 'sec_schema.underlying_table'
pga = db_info.get_db_info()


  sec_db


In [18]:
sql = f"""
select * from {futtab} where symbol='CLF21' and settle_date>='20200101'
"""
df_clf21 = pga.get_sql(sql)
df_clf21[(df_clf21.settle_date>=20200801) & (df_clf21.settle_date<=20201001)]

,symbol,settle_date,contract_num,open,high,low,close,adj_close,volume,open_interest
147,CLF21,20200803,5,41.42,42.41,40.91,42.28,42.28,15125,82282
148,CLF21,20200804,5,41.80,43.00,41.42,42.72,42.72,22750,84968
149,CLF21,20200805,5,42.45,44.32,42.39,43.31,43.31,35654,88455
150,CLF21,20200806,5,43.15,43.72,42.85,43.18,43.18,37404,89284
151,CLF21,20200807,5,43.16,43.35,42.34,42.48,42.48,24766,88614
152,CLF21,20200810,5,42.50,43.39,42.50,43.08,43.08,20562,89435
153,CLF21,20200811,5,43.05,43.93,42.67,42.80,42.80,20453,90087
154,CLF21,20200812,5,42.85,43.96,42.74,43.78,43.78,27665,91267
155,CLF21,20200813,5,43.69,43.91,43.30,43.44,43.44,30065,92499
156,CLF21,20200814,5,43.55,43.71,42.98,43.30,43.30,29618,97472


In [16]:
sql = f"""
select * from {opttab} where symbol='CLF21' and settle_date>='20200101' 
and strike = 50.0 and pc='P'
"""
df_clf21 = pga.get_sql(sql)
df_clf21[(df_clf21.settle_date>=20200801) & (df_clf21.settle_date<=20201001)]

,symbol,strike,pc,settle_date,open,high,low,close,adj_close,volume,open_interest
117,CLF21,50.0,P,20200803,0.0,10.04,8.84,8.91,8.91,0,901
118,CLF21,50.0,P,20200804,0.0,9.54,8.31,8.50,8.50,0,901
119,CLF21,50.0,P,20200805,0.0,8.58,7.35,7.98,7.98,0,901
120,CLF21,50.0,P,20200806,0.0,8.28,7.71,8.06,8.06,0,901
121,CLF21,50.0,P,20200807,0.0,8.67,8.01,8.58,8.58,0,901
122,CLF21,50.0,P,20200810,0.0,8.06,7.86,8.06,8.06,0,901
123,CLF21,50.0,P,20200811,0.0,8.36,7.45,8.28,8.28,0,901
124,CLF21,50.0,P,20200812,0.0,7.58,7.48,7.58,7.58,0,901
125,CLF21,50.0,P,20200813,0.0,7.90,7.46,7.83,7.83,0,901
126,CLF21,50.0,P,20200814,0.0,8.15,7.78,7.91,7.91,0,901


## Define methods to graph skew, and create subplots with separate legends.  

#### This only works with plotly 4..0 and above 

In [20]:
# # use this method to generate x/y scatter or bar graphs
# def plotly_plot(df_in,x_column,plot_title=None,
#                 y_left_label=None,y_right_label=None,
#                 bar_plot=False,figsize=(16,10),
#                 number_of_ticks_display=20,
#                 yaxis2_cols=None,
#                 x_value_labels=None):
#     '''
#     Return a plotly Figure that you can use with iplot, to produce
#     multi-axis x/y scatter and bar graphs
    
#     Example:
# xvals = list(range(1,101))
# y1vals = np.arange(101,201,1)
# y2vals = np.arange(11,1,-.1)
# y3vals = y2vals * .5
             
# df = pd.DataFrame({'my_x_vals':xvals, 
#                     'yvals_1':y1vals,
#                     'yvals_2':y2vals,
#                     'yvals_3':y3vals
#                   })
# fig = plotly_plot(df_in=df,x_column='my_x_vals',
#                     plot_title = "example graph",
#                     y_left_label='main y vals',
#                     y_right_label='alt y vals',
#                     yaxis2_cols = ['yvals_2','yvals_3'],
#                     number_of_ticks_display=25)
# iplot(fig)

    
#     :param df_in: data frame with a single x, and multiple y values to graph
#     :param x_column: the column in df_in that holds the x-axis values
#     :param plot_title: title of plot 
#     :param y_left_label:
#     :param y_right_label:
#     :param bar_plot: If True, plot bars, other wise, plot x/y scatter line
#     :param figsize: 
#     :param number_of_ticks_display: Number of x axis major ticks to display
#     :param yaxis2_cols: the dataframe columns that hold the y values for the second y axis, if any
#     :param x_value_labels: if you want alternate labels for the x axis, specify them here
#     '''
#     ya2c = [] if yaxis2_cols is None else yaxis2_cols
#     ycols = [c for c in df_in.columns.values if c != x_column]
#     # create tdvals, which will have x axis labels
#     td = list(df_in[x_column]) 
#     nt = len(df_in)-1 if number_of_ticks_display > len(df_in) else number_of_ticks_display
#     spacing = len(td)//nt
#     tdvals = td[::spacing]
#     tdtext = tdvals
#     if x_value_labels is not None:
#         tdtext = [x_value_labels[i] for i in tdvals]
    
#     # create data for graph
#     data = []
#     # iterate through all ycols to append to data that gets passed to go.Figure
#     for ycol in ycols:
#         if bar_plot:
#             b = go.Bar(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
#         else:
#             b = go.Scatter(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
#         data.append(b)

#     # create a layout
#     layout = go.Layout(
#         title=plot_title,
#         xaxis=dict(
#             ticktext=tdtext,
#             tickvals=tdvals,
#             tickangle=45,
#             type='category'),
#         yaxis=dict(
#             title='y main' if y_left_label is None else y_left_label
#         ),
#         yaxis2=dict(
#             title='y alt' if y_right_label is None else y_right_label,
#             overlaying='y',
#             side='right'),
#         margin=Margin(
#             b=100
#         )        
#     )

#     fig = go.Figure(data=data,layout=layout)
#     return fig

# def plotly_shaded_rectangles(beg_end_date_tuple_list,fig):
#     ld_shapes = []
#     for beg_end_date_tuple in beg_end_date_tuple_list:
#         ld_beg = beg_end_date_tuple[0]
#         ld_end = beg_end_date_tuple[1]
#         ld_shape = dict(
#             type="rect",
#             # x-reference is assigned to the x-values
#             xref="x",
#             # y-reference is assigned to the plot paper [0,1]
#             yref="paper",
#             x0=ld_beg[i],
#             y0=0,
#             x1=ld_end[i],
#             y1=1,
#             fillcolor="LightSalmon",
#             opacity=0.5,
#             layer="below",
#             line_width=0,
#         )
#         ld_shapes.append(ld_shape)

#     fig.update_layout(shapes=ld_shapes)
#     return fig

In [48]:
def figure_crosshairs(fig):
    ''' add crosshairs to plotly_plot figure
    '''
    fig['layout'].hovermode='x'
    fig['layout'].yaxis.showspikes=True
    fig['layout'].xaxis.showspikes=True
    fig['layout'].yaxis.spikemode="toaxis+across"
    fig['layout'].xaxis.spikemode="toaxis+across"
    fig['layout'].yaxis.spikedash="solid"
    fig['layout'].xaxis.spikedash="solid"
    fig['layout'].yaxis.spikethickness=1
    fig['layout'].xaxis.spikethickness=1
    fig['layout'].spikedistance=1000
    return fig


def plotly_plot(df_in,x_column,plot_title=None,
                y_left_label=None,y_right_label=None,
                bar_plot=False,width=800,height=400,
                number_of_ticks_display=20,
                yaxis2_cols=None,
                x_value_labels=None,
                modebar_orientation='v',modebar_color='grey',
                legend_x=None,legend_y=None,
                title_y_pos = 0.9,
                title_x_pos = 0.5):
    '''
    Plot and x/y graph
    '''
    
    ya2c = [] if yaxis2_cols is None else yaxis2_cols
    ycols = [c for c in df_in.columns.values if c != x_column]
    # create tdvals, which will have x axis labels
    td = list(df_in[x_column]) 
    nt = len(df_in)-1 if number_of_ticks_display > len(df_in) else number_of_ticks_display
    spacing = len(td)//nt
    tdvals = td[::spacing]
    tdtext = tdvals
    if x_value_labels is not None:
        tdtext = [x_value_labels[i] for i in tdvals]
    
    # create data for graph
    data = []
    # iterate through all ycols to append to data that gets passed to go.Figure
    for ycol in ycols:
        if bar_plot:
            b = go.Bar(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        else:
            b = go.Scatter(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        data.append(b)

    # create a layout

    layout = go.Layout(
        title=plot_title,
        xaxis=dict(
            ticktext=tdtext,
            tickvals=tdvals,
            tickangle=45,
            type='category'),
        yaxis=dict(
            title='y main' if y_left_label is None else y_left_label
        ),
        yaxis2=dict(
            title='y alt' if y_right_label is None else y_right_label,
            overlaying='y',
            side='right'),
        autosize=True,
#         autosize=False,
#         width=width,
#         height=height,
        margin=Margin(
            b=100
        ),
        modebar={'orientation': modebar_orientation,'bgcolor':modebar_color}
    )

    fig = go.Figure(data=data,layout=layout)
    fig.update_layout(
        title={
            'text': plot_title,
            'y':title_y_pos,
            'x':title_x_pos,
            'xanchor': 'center',
            'yanchor': 'top'})
    if (legend_x is not None) and (legend_y is not None):
        fig.update_layout(legend=dict(x=legend_x, y=legend_y))
    fig = figure_crosshairs(fig)
    return fig


def plotly_shaded_rectangles(beg_end_date_tuple_list,fig):
    '''
    Add shaded rectanges that highlight parts of an x/y graph
    '''
    ld_shapes = []
    for beg_end_date_tuple in beg_end_date_tuple_list:
        ld_beg = beg_end_date_tuple[0]
        ld_end = beg_end_date_tuple[1]
        ld_shape = dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0=ld_beg[i],
            y0=0,
            x1=ld_end[i],
            y1=1,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )
        ld_shapes.append(ld_shape)

    fig.update_layout(shapes=ld_shapes)
    return fig

In [21]:
xvals = list(range(1,101))
y1vals = np.arange(101,201,1)
y2vals = np.arange(11,1,-.1)
y3vals = y2vals * .5
             
df = pd.DataFrame({'my_x_vals':xvals, 
                    'yvals_1':y1vals,
                    'yvals_2':y2vals,
                    'yvals_3':y3vals
                  })
fig = plotly_plot(df_in=df,x_column='my_x_vals',
                    plot_title = "example graph",
                    y_left_label='main y vals',
                    y_right_label='alt y vals',
                    yaxis2_cols = ['yvals_2','yvals_3'],
                    number_of_ticks_display=25)
iplot(fig)



In [22]:
def create_skew_per_date_df(df):
    '''
    Find the first settle_date whose count of rows is equal to max count of rows.
    '''
    # get the first symbol (which should be the only symbol)
    contract = df.symbol.unique()[0]
    # get just that symbol's data
    df12 = df[df.symbol==contract]
    df_counts = df12[['settle_date','moneyness']].groupby('settle_date',as_index=False).count()
    max_count = df_counts.moneyness.max()
    first_max_count_settle_date = df_counts[df_counts.moneyness==max_count].iloc[0].settle_date
    
    df_ret = df12[df12.settle_date==first_max_count_settle_date][['moneyness']]
    all_settle_dates = sorted(df_counts.settle_date.unique())
    for settle_date in all_settle_dates:
        df_temp = df12[df12.settle_date==settle_date][['moneyness','vol_skew']]
        df_ret = df_ret.merge(df_temp,on='moneyness',how='outer')
        df_ret = df_ret.rename(columns={'vol_skew':str(settle_date)})
    df_ret = df_ret.sort_values('moneyness')
    df_ret.moneyness = df_ret.moneyness.round(4)
    return df_ret


In [23]:
def graph_skew(df_iv_final,do_plot=False):
    '''
    Graph skew for ONLY ONE symbol.
    If df_iv_final contains more than one symbol, we will only graph the first symbol in the DataFrames    
    '''
    # get the first symbol (which should be the only symbol)
    contract = df_iv_final.symbol.unique()[0]
    # get just that symbol's data and only days that have sufficient skew data
    dft = df_iv_final[df_iv_final.symbol==contract]
    dft_count = dft[['settle_date','symbol']].groupby('settle_date',as_index=False).count()
    valid_settle_dates = dft_count[dft_count.symbol>2].settle_date.unique()
    dft = dft[dft.settle_date.isin(valid_settle_dates)]
    dfp = create_skew_per_date_df(dft)
    
    settle_dates = sorted([c for c in dfp.columns.values if c != 'moneyness'])
    splits = list(np.arange(5,len(settle_dates),5))
    settle_date_groups = np.split(np.array(settle_dates),splits)
    ret_figs = []
    for sdg in settle_date_groups:
        sdg_sorted = [str(c) for c in sorted(sdg)]
        cols = ['moneyness']+list(sdg_sorted)
        dfp_sub = dfp[cols]
        t = f'{contract} {sdg[0]} - {sdg[-1]}' 
        f = plotly_plot(dfp_sub,x_column='moneyness',plot_title=t,y_left_label='vol skew')
        ret_figs.append(f)
        if do_plot:
            iplot(f)
    return ret_figs

#### Do the same plots as above, but using a grid

In [24]:

def graph_skew_subplot_quad(fig_group,rows=1,cols=2):
    '''
    Use subplots to output the results of the method graph_skew above
    '''
    f1 = make_subplots(rows=rows, cols=cols,  
        shared_yaxes=False,
        shared_xaxes=False,                       
        subplot_titles=[fig_group[i]['layout'].title.text for i in range(len(fig_group))],
        horizontal_spacing=0.05,
        vertical_spacing=0.11,                       
        print_grid=False)

    pl_width=450*cols 
    pl_height=400*rows
    title = 'Skew plots<br>'

    f1.update_layout(title=title,                                 
        font= Font(family="Open Sans, sans-serif"),
        showlegend=True,     
        hovermode='x',  
        autosize=True,       
        width=pl_width,       
        height=pl_height,
        plot_bgcolor='#EFECEA', 
        bargap=0.05,
        margin=Margin(
                      l=5,
                      r=5,
                      b=55,
                      t=50
        )
    )    

    for i in range(len(fig_group)):
        x = int(i/2) + 1
        y = i % 2 + 1
        f = fig_group[i]
        l = f.layout
        if y > 1:
            l.yaxis.title=''
        try:
            yaxis = f'yaxis{i+1}'
            xaxis = f'xaxis{i+1}'
            if i < 10:
                yaxis = yaxis.replace('1','') 
                xaxis = xaxis.replace('1','') 
            gname = f'{x,y}'#rfs[i]['layout'].title
            for d in f.data:
                data_y = f'y{i+1}'.replace('1','') 
                d['yaxis']=data_y
                d['legendgroup'] =  gname
                d['name'] = f"{d.name}"
                f1.add_trace(d,x,y)
                f1.update_xaxes(patch=l.xaxis,row=x,col=y)
                f1.update_yaxes(patch=l.yaxis,row=x,col=y)                
        except Exception as e:
            print(f'graph_skew_subplots ERRORS: {str(e)}')
    return f1


def graph_skew_subplots(df,rows=2,cols=2):
    fig_list = graph_skew(df)
    n = rows*cols   
    # using list comprehension 
    fig_groups = [fig_list[i*n:(i + 1)*n] for i in range((len(fig_list) + n - 1) // n )]  
    for fig_group in fig_groups:
        iplot(graph_skew_subplot_quad(fig_group,rows=rows,cols=cols))


### Graph skew changes historically, per day

In [32]:
SYMBOL_TO_RESEARCH = 'CL'
df_iv_final = pd.read_csv(f'./temp_folder/df_iv_final_{SYMBOL_TO_RESEARCH}.csv')
df_iv_skew = pd.read_csv(f'./temp_folder/df_iv_skew_{SYMBOL_TO_RESEARCH}.csv')
# all_contracts = pga.get_sql(f"select symbol from {opttab} where substring(symbol,1,2)='{SYMBOL_TO_RESEARCH}'").symbol.unique()
all_contracts = df_iv_final[df_iv_final.symbol.str.slice(0,2)==SYMBOL_TO_RESEARCH].symbol.unique()
all_contracts 

array(['CLH11', 'CLJ11', 'CLK11', 'CLM11', 'CLN11', 'CLQ11', 'CLU11',
       'CLV11', 'CLX11', 'CLZ11', 'CLF12', 'CLG12', 'CLH12', 'CLJ12',
       'CLK12', 'CLM12', 'CLN12', 'CLQ12', 'CLU12', 'CLV12', 'CLX12',
       'CLZ12', 'CLF13', 'CLG13', 'CLH13', 'CLJ13', 'CLK13', 'CLM13',
       'CLN13', 'CLQ13', 'CLU13', 'CLV13', 'CLX13', 'CLZ13', 'CLF14',
       'CLG14', 'CLH14', 'CLJ14', 'CLK14', 'CLM14', 'CLN14', 'CLQ14',
       'CLU14', 'CLV14', 'CLX14', 'CLZ14', 'CLF15', 'CLG15', 'CLH15',
       'CLJ15', 'CLK15', 'CLM15', 'CLN15', 'CLQ15', 'CLU15', 'CLV15',
       'CLX15', 'CLZ15', 'CLF16', 'CLG16', 'CLH16', 'CLJ16', 'CLK16',
       'CLM16', 'CLN16', 'CLQ16', 'CLU16', 'CLV16', 'CLX16', 'CLZ16',
       'CLF17', 'CLG17', 'CLH17', 'CLJ17', 'CLK17', 'CLM17', 'CLN17',
       'CLQ17', 'CLU17', 'CLV17', 'CLX17', 'CLZ17', 'CLF18', 'CLG18',
       'CLH18', 'CLJ18', 'CLK18', 'CLM18', 'CLN18', 'CLQ18', 'CLU18',
       'CLV18', 'CLX18', 'CLZ18', 'CLF19', 'CLG19', 'CLH19', 'CLJ19',
       'CLK19', 'CLM

In [33]:
grid_plot=True
clist = [c for c in all_contracts if (c[:2]==f'{SYMBOL_TO_RESEARCH}') & (c[-2:]=='21')]
volcols = ['symbol','settle_date','moneyness','vol_skew']
dfivf = df_iv_final[volcols].groupby(['symbol','settle_date','moneyness'],as_index=False).mean()

for c in clist:
    dft = dfivf[dfivf.symbol==c]
    if len(dft)<=0:
        print(f'no data for symbol {c}')
        continue
    if grid_plot:
        graph_skew_subplots(dfivf[dfivf.symbol==c],rows=1,cols=2)
    else:
        rls = graph_skew(dfivf[dfivf.symbol==c],do_plot=True)
        

### Use the implied vol skew dataframes to graph changes in vol skew vs price

In [36]:
y = 20
syms = [s for s in all_contracts if  (int(s[-2:]) >= y)]

for sym in syms:
    df_fut = pga.get_sql(f"select symbol,settle_date,close from {futtab} where symbol = '{sym}'")
    df_year = df_iv_skew[df_iv_skew.symbol==sym]
    df_year['2dn_2up'] = df_year['-0.2'] - df_year['0.2']
    df_year = df_year[['settle_date','2dn_2up']]
    df_year = df_year.merge(df_fut[['settle_date','close']],on='settle_date',how='inner')
    iplot(plotly_plot(df_year,x_column='settle_date',
            yaxis2_cols=['close'],y_left_label='skew',y_right_label=sym,
                     plot_title=f'{sym} close vs skew -0.2, 0.2',
                     number_of_ticks_display=15))
    

/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [35]:
y = 20
syms = [s for s in all_contracts if  (int(s[-2:]) >= y)]

for sym in syms:
    df_fut = pga.get_sql(f"select symbol,settle_date,close from {futtab} where symbol = '{sym}'")
    df_year = df_iv_final[['settle_date','atm_iv']][df_iv_final.symbol==sym]
    df_year = df_year.groupby(['settle_date'],as_index=False).mean()
    df_year.atm_iv = df_year.atm_iv.round(4)
    df_year = df_year.merge(df_fut[['settle_date','close']],on='settle_date',how='inner')
    iplot(plotly_plot(df_year,x_column='settle_date',
            yaxis2_cols=['close'],y_left_label='atm_iv',y_right_label=sym,
                     plot_title=f'{sym} close vs atm implied vol',
                     number_of_ticks_display=15))
    

### Create a constant maturety contract, and show it's skew over time

,settle_date,2dn_2up,symbol,close
131,20200501,0.4909,CLZ99,19.69
132,20200504,0.4187,CLZ99,20.39
133,20200505,0.3971,CLZ99,25.28
134,20200506,0.4554,CLZ99,24.09
135,20200507,0.4432,CLZ99,23.55
...,...,...,...,...
274,20201123,0.2099,CLZ99,43.06
275,20201124,0.1788,CLZ99,44.91
276,20201125,0.1716,CLZ99,45.86
277,20201127,0.1716,CLZ99,45.53


In [52]:
y = 20
syms = [s for s in all_contracts if  (int(s[-2:]) >= y)]
df_all_years = None
for sym in syms:
    df_year = df_iv_skew[df_iv_skew.symbol==sym]
    df_year['2dn_2up'] = df_year['-0.2'] - df_year['0.2']
    df_year = df_year[['settle_date','2dn_2up']]
    if df_all_years is None:
        df_all_years = df_year.copy()
    else:
        df_all_years = df_all_years.append(df_year,ignore_index=True)

df_all_years = df_all_years.sort_values('settle_date')
df_all_years = df_all_years.rename(columns={'2dn_2up':'20% up/down Skew Diff','close':'CL Closing Price'})
df_all_years = df_all_years[['settle_','20% up/down Skew Diff','CL Closing Price']]
first_settle_date = df_all_years.settle_date.min()
last_settle_date = df_all_years.settle_date.max()

prod = syms[0][0:2]
sql_99 = f"""
select symbol,settle_date,close from {futtab} 
where symbol = '{prod}Z99' 
and settle_date>= {first_settle_date}
and settle_date<={last_settle_date} 
"""

df_fut = pga.get_sql(sql_99)
df_all_years = df_all_years.merge(df_fut,on='settle_date',how='inner')

# only look from 05/01/2020 on b/c of the vol spike in April
df_all_years = df_all_years[df_all_years.settle_date>=20200501]

# create the figure
fig = plotly_plot(df_all_years,x_column='settle_date',
        yaxis2_cols=['close'],y_left_label='skew',y_right_label='CL Cash Price',
                 plot_title=f'CL Cash close vs skew -0.2, 0.2',
                 number_of_ticks_display=15)
# display it
iplot(fig)
# write it to an html file
fig.write_html('temp_folder/cl_cash_vs_volskew.html')

/Users/bperlman1/Virtualenvs3/dashrisk5/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [14]:
def get_postgres_data(contract,pga):
    osql = f"select * from {opttab} where symbol='{contract}';"
    dfo = pga.get_sql(osql)
    usql = f"select * from {futtab} where symbol='{contract}';"
    dfu = pga.get_sql(usql)
    # Merge options and futures data
    df = dfo.merge(dfu,how='inner',on=['symbol','settle_date'])
    # Get options expiration dates
    df_expiry_dates = dfo[['symbol','settle_date']].groupby('symbol',as_index=False).max()
    return df,df_expiry_dates

In [37]:
USE_PYVOL = True
def lam_pyvol(r):
    try:
        return implied_volatility.implied_volatility(r.close_x,r.close_y,r.strike,.02,r.dte/365, r.pc.lower())
    except:
        return -1
# lam_pyvol = lambda r:implied_volatility.implied_volatility(r.close_x,r.close_y,r.strike,.02,r.dte/365, r.pc.lower())
lam_mibian = lambda r:mibian.BS([r.close_y,r.strike,2,r.dte], callPrice=r.close_x).impliedVolatility

def get_implieds(df,df_expiry_dates,contract):
    df2 = df[['symbol','contract_num','pc','settle_date','strike','close_x','close_y']]
    df2 = df2[(((df2.pc=='C' )& (df2.strike>=df2.close_y)) | ((df2.pc=='P' ) & (df2.strike<df2.close_y)))  & (df2.symbol.str.contains(contract))]
    df2 = df2[df2.contract_num==2]
    phigh = df2.close_y.max()
    plow = df2.close_y.min()
    high_strike = round(phigh * 1.3)
    low_strike = round(plow * .7)
    df2 = df2[(df2.strike>=low_strike) & (df2.strike<=high_strike)]

    df9 = df2[df2.symbol==contract]
    df9 = df9.merge(df_expiry_dates.rename(columns={'settle_date':'expiry'}),on='symbol',how='inner')
    df9['syear'] = df9.settle_date.astype(str).str.slice(0,4).astype(int)
    df9['smon'] = df9.settle_date.astype(str).str.slice(4,6).astype(int)
    df9['sday'] = df9.settle_date.astype(str).str.slice(6,8).astype(int)
    df9['eyear'] = df9.expiry.astype(str).str.slice(0,4).astype(int)
    df9['emon'] = df9.expiry.astype(str).str.slice(4,6).astype(int)
    df9['eday'] = df9.expiry.astype(str).str.slice(6,8).astype(int)
    df9['sdatetime'] = df9.apply(lambda r:datetime.datetime(r.syear,r.smon,r.sday),axis=1)
    df9['edatetime'] = df9.apply(lambda r:datetime.datetime(r.eyear,r.emon,r.eday),axis=1)
    df9['dte'] = df9.edatetime - df9.sdatetime
    df9.dte = df9.dte.dt.days
    df9 = df9[['symbol','settle_date','pc','contract_num','strike','close_x','close_y','dte']]
    df10 = df9.iloc[:len(df9)].copy()
    df10.index = list(range(len(df10)))
    if USE_PYVOL:
        df10['iv'] = df10.apply(lam_pyvol,axis=1)
    else:
        n = 100
        for i in tqdm_notebook(np.arange(0,len(df10)-n,n)):
                df10.loc[i:i+n,'iv'] = df10.loc[i:i+n].apply(lam_mibian,axis=1)
        print(f'doing remaining {datetime.datetime.now()}')
        i = df10[df10.iv.isna()].index[0]
        df10.loc[i:,'iv'] = df10.loc[i:].apply(lam_mbian,axis=1)
        print(f'done with remaining {datetime.datetime.now()}')
    return df10



In [38]:
def get_even_moneyness_strikes(df10):
    # define amounts around the money which will help create strikes to add
    moneyness = np.arange(.7,1.4,.05).round(6)
    # define columns on which to execute groupby
#     gb_cols = ['symbol','settle_date','pc','contract_num','dte','close_y']
    gb_cols = ['symbol','settle_date','contract_num','dte','close_y']
    # define function used in groupby.apply to create strikes and iv's at those strikes
    #   where the strikes are an even amount from the money 
    #   (like .7, .8, ... 1, 1.1, 1.2, etc)
    def _add_even_moneyness_strikes(df):
        # get underlying from first row (the groupby makes them all the same)
        r = df.iloc[0]
        underlying = r.close_y
        # create new rows to append to df, using only the gb_cols
        df_ret1 = df.iloc[:len(moneyness)][gb_cols].copy()
        # add nan iv's !!!! MUST BE np.nan - NOT None
        df_ret1['iv'] = np.nan
        # add new strikes
        df_ret1['strike'] = moneyness * underlying
        # append the new strikes
        dfa = df.append(df_ret1,ignore_index=True,sort=True).copy()
        df_ret2 = dfa.sort_values(['symbol','settle_date','pc','strike'])
        df_ret2 = df_ret2.drop_duplicates(subset='strike')
        # set the index to the strike so that interpolate works
        df_ret2.index = df_ret2.strike
        # create interpolated iv's
        df_ret2['iv'] = df_ret2.iv.interpolate(method='polynomial', order=2)
        # reset the index
        df_ret2.index = list(range(len(df_ret2)))
        return df_ret2

    # start here
    df11 = df10.groupby(gb_cols).apply(_add_even_moneyness_strikes).copy()
    df11.index = list(range(len(df11)))
    df11['moneyness'] = df11.strike / df11.close_y
    df11.moneyness = df11.moneyness.round(4)

    df12 = df11[(df11.moneyness.isin(moneyness)) & (~df11.iv.isna())].copy()
    df12.moneyness  = df12.moneyness - 1
    df12.index = list(range(len(df12)))
    df12_atm = df12[df12.moneyness==0][['symbol','settle_date','pc','iv']]
    df12_atm = df12_atm.rename(columns={'iv':'atm_iv'})
    
    df12_atm = df12_atm.drop_duplicates()
    df12 = df12.merge(df12_atm,on=['symbol','settle_date','pc'],how='inner')
    df12.moneyness = df12.moneyness.round(4)
    df12['vol_skew'] = (df12.iv - df12.atm_iv).round(4)
    return df12



## END